In [54]:
#install required packages
!pip install transformers

In [128]:
import torch
import torch.nn as nn
# Define a custom neural network class
class ConvertModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(ConvertModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        x = x.view(N, -1)  # Reshape the input tensor
        x = self.linear(x)
        return x

# Create an instance of the custom class
def tensor_3d_to_2d(tensor):
  N = tensor.shape[0]
  M = tensor.shape[1]
  input_dim = M * 768
  output_dim = 768
  tensor_model = ConvertModel(input_dim, output_dim)
  converted_tensor = tensor_model(tensor)

  return converted_tensor




In [127]:
# Apply the model to convert the tensor
# Create an N x M x 768 tensor
N = 10  # Replace with your desired value of N
M = 5   # Replace with your desired value of M
tensor_NxMx768 = torch.randn(N, M, 768)
converted_tensor = model(tensor_NxMx768)

print(converted_tensor.shape)

torch.Size([10, 768])


In [120]:
#add the Neural Network layer to convert the toensor size from N*M*768 to N*768
def tensor_to_NN(tensor_3d):
  tensor_2d = tensor_3d.reshape(tensor_3d.shape[0], -1).numpy()
  result = tensor_2d

  return result

In [2]:
#test GPU device
import tensorflow as tf
tf.test.gpu_device_name()
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 8258828640572734106
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14328594432
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 3645499678627768707
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [55]:
#add bert model
from transformers import BertTokenizerFast, BertForTokenClassification
from torch.nn.functional import softmax
import torch
#import packages
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import precision_score, recall_score


In [56]:
#import dataset
# test_df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)
#import dataset from github
ADETrainingdata = pd.read_csv('https://raw.githubusercontent.com/FANMISUA/TweetAENormalization/main/ADEClassification/ADETraining.tsv', delimiter='\t')
ADEValidationdata = pd.read_csv('https://raw.githubusercontent.com/FANMISUA/TweetAENormalization/main/ADEClassification/ADEValidation.tsv', delimiter='\t')

In [5]:
#import normalization data
RawTweets = pd.read_csv('https://raw.githubusercontent.com/FANMISUA/TweetAENormalization/main/ADENormalization/Dev/tweets.tsv',delimiter='\t')
SpansNorm = pd.read_csv('https://raw.githubusercontent.com/FANMISUA/TweetAENormalization/main/ADENormalization/Dev/spans_norm.tsv',delimiter='\t')

#check the tweets id in Spans Norm and Raw Tweets
# print("get the tweets id in Raw Tweets:\n",RawTweets.iloc[:,0])
# print("Get the tweets id in Spans Norm:\n",SpansNorm.iloc[:,0])
list1 = RawTweets.iloc[:,0]
list2 = SpansNorm.iloc[:,0]
common_values = set(list1).intersection(set(list2))
# print("common tweets id:",common_values)
# print("Total tweets:",len(set(list1)),"Tweets with AE:",len(set(list2)),"matched tweets:",len(common_values))

In [57]:
def proprocess_tweets(tweets):
  processed_tweets = [preprocessTweet1(tweet) for tweet in tweets]
  return processed_tweets
#add preprocessing step to clean the tweets
import re
def preprocessTweet1(tweet):
    # process the tweets

    # Remove 'RT' from tweet
    tweet = re.sub(r'RT[\s]+', '', tweet)
    # Remove &amp; (ampersand)
    tweet = re.sub(r'&amp;', 'and', tweet)
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',tweet)
    #Convert @username to @USER
    tweet = re.sub('@[^\s]+','',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    # Remove all emojis.
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)

    tweet = emoji_pattern.sub(r'', tweet)

    # Remove redundant spaces
    tweet = re.sub(r'\s+', ' ', tweet).strip()

    #remove unknown characters
    tweet = ''.join([c for c in tweet if ord(c) < 128])

    #trim
    tweet = tweet.strip('\'"')
    return tweet



In [58]:
# Define a custom neural network class
class ConvertModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(ConvertModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        x = x.view(1, -1)  # Reshape the input tensor
        x = self.linear(x)
        return x

#convert n*768 tensor to 768 tensor
def tensortoNN(tensor,n):
  input_dim = 768 * n
  output_dim = 768
  NNmodel = ConvertModel(input_dim, output_dim)
  converted_tensor = NNmodel(tensor)

  return converted_tensor

In [59]:
# train the ADE model
#print(ADETrainingdata.head() , ADEValidationdata.head())
#use the first 2000 rows of the training data
# ADETrainingdata[:2000]
# ADETrainingdata.iloc[:3,-3:]
#ADETrainingdata['tweet_id'][:200]
#only use the first 300 tweets in Training data as training
print(ADETrainingdata['label'][:300].value_counts())
training_batch_1 =ADETrainingdata[:300]
print(training_batch_1.head())

#testing dataset from validation dataset
#use first 100 for coding
print(ADEValidationdata['label'][:100].value_counts())
testing_batch_1 =ADEValidationdata[:100]
print(testing_batch_1.head())

0    255
1     45
Name: label, dtype: int64
              tweet_id                                               text  \
0  1382343793341575169  @IrvineWelsh I don’t know about you Irvine but...   
1  1377631738692796417  I bet money if i went n took a covid test righ...   
2  1386448010029240326  @JamesMelville My wife received a POSITIVE Cov...   
3  1361342676340211717  Out of the 180,000+ people who have had the tw...   
4  1386757983254765569  My whole family is sick af and here I am now i...   

   label  
0      0  
1      0  
2      0  
3      0  
4      0  
0    88
1    12
Name: label, dtype: int64
              tweet_id                                               text  \
0  1384271458210238470  4 possible 8 Michigan Republicans test positiv...   
1  1379153840780546053  @Rica_DoesIt That’s the only indication that I...   
2  1387196609147006976  @chrsdcook @thehill I didn’t even know I had c...   
3  1378098447413350407  I hope two of the people attending don't test ...   


In [109]:
#define the tokenize, padding, masking procedure from text to tensor

def text_to_tensor(data):
  #tokenize
  tokenized = data['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
  #padding
  max_len = 0
  for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
  padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
  print("max len for padding: ",max_len, np.array(padded).shape)
  #Masking the paddding
  attention_mask = np.where(padded != 0, 1, 0)
  # print("attention mask shape:",attention_mask.shape)
  #Use pretrained Bert model to word embedding all the sentences
  input_ids = torch.tensor(padded)
  attention_mask = torch.tensor(attention_mask)

  with torch.no_grad():
      last_hidden_states = model(input_ids, attention_mask=attention_mask)
  # result =last_hidden_states

  return last_hidden_states[0]

In [60]:
# clean data
def clean_data(data):
  #clean tweets
  data['text'] = proprocess_tweets(data['text'])
  return(data)


In [61]:

#balance the data
def balanced_data(data):

  data_ade = data[data['label'] == 1]
  data_noade = data[data['label'] == 0]
  # print(data_ade.shape,data_noade.shape)
  #downsampling
  data_noade_downsampled = data_noade.sample(data_ade.shape[0])
  data_balanced = pd.concat([data_ade,data_noade_downsampled])
  # print(t_batch_balanced['label'].value_counts())
  #proprecess the tweets
  # print("before clean",t_batch_balanced)
  # #clean tweets
  # data_balanced['text'] = proprocess_tweets(data_balanced['text'])
  # print("after clean  ",data_balanced)
  return data_balanced



In [70]:
# proprecess training data
# print(training_batch_1.shape)
# print(balanced_data(training_batch_1).shape)
# print(training_batch_1)
#balance data
bal_training_data = balanced_data(training_batch_1)
#clean data
clean_training_data = clean_data(bal_training_data)
# print(bal_data)
print("training data shape:",training_batch_1.shape,"clean balanced training data shape",clean_training_data.shape)

# proprecess testing data
#all clean testing data unbalanced
all_testing_data = clean_data(testing_batch_1)
#balance testing data
bal_testing_data = balanced_data(testing_batch_1)
#clean balanced testing data
clean_testing_data = clean_data(bal_testing_data)

print("testing data shape:",testing_batch_1.shape,"all clean testing data shape:",all_testing_data.shape,
      "clean balanced testing data shape:",clean_testing_data.shape)


training data shape: (300, 3) clean balanced training data shape (90, 3)
testing data shape: (100, 3) all clean testing data shape: (100, 3) clean balanced testing data shape: (24, 3)


In [86]:
#split training and testing
train_x, test_x, train_y, test_y = train_test_split(clean_training_data['text'], clean_training_data['label'],  test_size=0.2, random_state=42)
# test_x.shape
print(train_x.shape,test_x.shape)
ext_ba_test_x, ext__ba_test_y = clean_testing_data['text'], clean_testing_data['label']
print(ext_test_y.shape)
ext_all_test_x, ext_all_test_y = all_testing_data['text'], all_testing_data['label']
print(ext_all_test_x.shape)

(72,) (18,)
(24,)
(100,)


In [87]:
#load bert model. load tokenizer and model
#Testing small bert model 'distilbert-base-uncased'
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
#testing pubmed bert model
# model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'ml4pubmed/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext_pub_section')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [114]:
# print(clean_training_data)
fe_train_1 = text_to_tensor(clean_training_data)
fe_test_1 = text_to_tensor(clean_testing_data)
fe_test_2 = text_to_tensor(all_testing_data)


max len for padding:  83 (90, 83)
max len for padding:  76 (24, 76)
max len for padding:  82 (100, 82)


In [115]:
#get the feature dimensions for the tensor
print(fe_train_1.shape,fe_test_1.shape,fe_test_2.shape)

torch.Size([90, 83, 768]) torch.Size([24, 76, 768]) torch.Size([100, 82, 768])


In [131]:
# print(fe_train_1.shape)
# print(tensor_to_NN(fe_train_1).shape)
# print(tensor_3d_to_2d(fe_train_1).shape)

torch.Size([90, 83, 768])
(90, 63744)


RuntimeError: ignored

In [1]:
print(fe_train_1.shape)
print(tensor_to_NN(fe_train_1).shape)
print(tensor_3d_to_2d(fe_train_1).shape)

NameError: ignored

In [ ]:
# #Get the tensor dimension
# print(last_hidden_states[0])
# print(last_hidden_states[0].size())
# #first position is [CLS] token.
# features = last_hidden_states[0][:,0,:].numpy()
# print(features)
# print(features.shape)

tensor([[[ 0.2829,  0.1567,  0.2272,  ..., -0.1869,  0.6070,  0.3700],
         [ 0.6401, -0.0036,  0.4816,  ...,  0.0065,  0.3466,  0.0479],
         [ 0.0592, -0.1083,  0.4402,  ..., -0.3974, -0.1379,  0.2461],
         ...,
         [ 0.5154,  0.2809,  0.0904,  ..., -0.2032,  0.3637,  0.0181],
         [ 0.5656,  0.1848,  0.1909,  ..., -0.0981,  0.3749, -0.2579],
         [ 0.4555,  0.0392,  0.1548,  ...,  0.0069,  0.2053, -0.0969]],

        [[ 0.0924,  0.1797,  0.0927,  ..., -0.1305,  0.4414,  0.4691],
         [ 0.4584,  0.1091,  0.2552,  ..., -0.0335,  0.4750,  0.5256],
         [ 1.0446, -0.2030,  0.2586,  ..., -0.5464,  0.1183,  0.2701],
         ...,
         [ 0.1503,  0.1455,  0.2511,  ..., -0.0866, -0.1659,  0.2501],
         [-0.0349,  0.2445,  0.2170,  ..., -0.1941,  0.0664,  0.1888],
         [ 0.1751,  0.0580,  0.3602,  ..., -0.0849, -0.0144,  0.2471]],

        [[-0.0798, -0.2039,  0.0721,  ..., -0.2190,  0.3414,  0.4212],
         [ 0.4364, -0.1264,  0.3858,  ...,  0

In [ ]:
#first position is [CLS] token.
#convert 3D tensor to 2D tensor
tensor_3d = last_hidden_states[0]

# Convert 3D tensor to 2D tensor
tensor_2d = tensor_3d.reshape(tensor_3d.shape[0], -1).numpy()

print("Original 3D tensor shape:", tensor_3d.shape)
print("Converted 2D tensor shape:", tensor_2d.shape)
# print(tensor_2d)
features = tensor_2d
print("features dimension:",features.shape)

#label indicating which sentence is positive and negative.
labels = t_batch_balanced['label']
# print(labels)
print("label dimension:",labels.shape)

#use classification models

train_features, test_features, train_labels, test_labels = train_test_split(features, labels,  test_size=0.2, random_state=42)
print("training feature dimension",train_features.shape)
print("testing feature dimension",test_features.shape)
print("training label dimension",train_labels.shape)
print("training label dimension",test_labels.shape)


Original 3D tensor shape: torch.Size([90, 83, 768])
Converted 2D tensor shape: (90, 63744)
features dimension: (90, 63744)
label dimension: (90,)
training feature dimension (72, 63744)
testing feature dimension (18, 63744)
training label dimension (72,)
training label dimension (18,)


In [ ]:
# #split the training and testing data
# #240 tweets as training; 60 tweets as testing
# train_features, test_features, train_labels, test_labels = train_test_split(features, labels,  test_size=0.2, random_state=42)
# print(train_features.shape)
# print(test_features.shape)
# print(train_labels.shape)
# print(test_labels.shape)


(240, 768)
(60, 768)
(240,)
(60,)


In [ ]:
#refine parameters for logisticRegression model
parameters = {'C': np.linspace(0.0001, 100, 20)}
grid_search = GridSearchCV(LogisticRegression(), parameters)
grid_search.fit(train_features, train_labels)

print('best parameters: ', grid_search.best_params_)
print('best scrores: ', grid_search.best_score_)

best parameters:  {'C': 5.263252631578947}
best scrores:  0.7219047619047619


In [ ]:
lr_clf = LogisticRegression(C=5.2)
lr_clf.fit(train_features, train_labels)

LogisticRegression(C=5.2)

In [ ]:
lr_clf.score(test_features, test_labels)

0.8888888888888888

In [ ]:
lr_clf.score(train_features, train_labels)

1.0

In [ ]:
from sklearn.metrics import precision_recall_curve, accuracy_score, f1_score

predict_test_labels = lr_clf.predict(test_features)
precision = precision_score(test_labels, predict_test_labels)
recall = recall_score(test_labels, predict_test_labels)
accuracy =accuracy_score (test_labels, predict_test_labels)
f1 = f1_score(test_labels, predict_test_labels)


print("Precision: ", precision)
print("Recall: ", recall)
print("accuracy:",accuracy)
print("F1:",accuracy)

precision, recall , thresholds = precision_recall_curve(test_labels, predict_test_labels)
thresholds

Precision:  1.0
Recall:  0.8333333333333334
accuracy: 0.8888888888888888
F1: 0.8888888888888888


array([0, 1])

In [ ]:
# print(test_labels.shape)
# print(predict_test_labels.shape)
tp = np.sum((predict_test_labels == 1) & (test_labels == 1))
tn = np.sum((predict_test_labels == 0) & (test_labels == 0))
fp = np.sum((predict_test_labels == 0) & (test_labels == 1))
fn = np.sum((predict_test_labels == 1) & (test_labels == 0))


print(test_labels)
print(predict_test_labels)
print("tp:",tp,"tn:",tn,"fp:",fp,"fn:",fn)

282    1
148    1
124    0
31     0
6      1
187    1
280    1
63     0
60     1
299    1
201    0
241    1
145    0
151    0
70     1
22     1
121    1
197    1
Name: label, dtype: int64
[1 0 0 0 1 1 0 0 1 1 0 1 0 0 1 1 1 1]
tp: 10 tn: 6 fp: 2 fn: 0


In [ ]:
#test code with neural network
import torch
from transformers import BertTokenizer, BertModel
import torch.nn as nn

# Load pre-trained BERT model and tokenizer
bert_model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Input text
text = "Hello, how are you?"

# Tokenize and encode text
inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
input_ids = inputs['input_ids']

# Get BERT embeddings
with torch.no_grad():
    bert_output = bert_model(input_ids)

# Extract embeddings
bert_embeddings = bert_output.last_hidden_state
print(bert_embeddings.shape)
tensor_3dbert = bert_embeddings

# Convert 3D tensor to 2D tensor
tensor_2dbert = tensor_3dbert.reshape(tensor_3dbert.shape[0], -1).numpy()
print(tensor_2dbert.shape)
# Define a simple neural network
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        return x

# Instantiate the neural network
input_size = bert_embeddings.shape[2]  # BERT embedding size
hidden_size = 64
output_size = 2  # Example: binary classification

net = NeuralNetwork(input_size, hidden_size, output_size)

# Use BERT embeddings as input to the neural network
output = net(bert_embeddings)
print(output.shape)


torch.Size([1, 8, 768])
(1, 6144)
torch.Size([1, 8, 2])


In [ ]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.846 (+/- 0.02)


In [ ]:
from transformers import pipeline
print(pipeline('sentiment-analysis')('I hate you'))


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


[{'label': 'NEGATIVE', 'score': 0.9991129040718079}]
